In [ ]:
# %pylab nbagg
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

from lab_repo.classes.dbclasses import dbExperimentSet, dbExperiment
from lab_repo.classes.classes import pcExperimentGroup

import numpy as np
import scipy
from scipy.stats import pearsonr, spearmanr, binned_statistic, sem, f_oneway, ttest_rel, ttest_ind, wilcoxon, mannwhitneyu,
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols

import pandas as pd

from datetime import datetime

import lab_repo.analysis.dendrites_analysis as da
from lab_repo.misc.plotting import cdf


import cPickle as pkl
import os

import lab_repo.misc.plotting as lpl

from matplotlib import rc

# Set rc-params
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['axes.labelsize'] = 7
plt.rcParams['boxplot.boxprops.linewidth'] = 1

rc('font',**{'family':'sans-serif','sans-serif':['helvetica', 'sans-serif']})
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def initializeExperimentGroup():

    pc_kwargs = {'imaging_label': 'mergedmerged',
                 'nPositionBins': 100,
                 'channel': 'Ch2',
                 'demixed': False,
                 'pf_subset': None,
                 'signal': 'spikes'}

    path = '/home/sebi/grps/dend_grp.json'
    eset = dbExperimentSet(project_name='sebi')
    grp = pcExperimentGroup.from_json(path, eset, **pc_kwargs)

    return grp

def group_order(x):
    
    if x['order'] < 7:
        return x['order']
    else:
        return 7
    
def clean(ax, full=False, offset=0):
    if full:
        sns.despine(ax=ax, left=True, bottom=True, offset=offset)
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        sns.despine(ax=ax, offset=offset)

In [ ]:
grp = initializeExperimentGroup()
grp = pcExperimentGroup([x for x in grp if x.condition==1], **grp.args)
colors = sns.xkcd_palette(['red orange', 'black', 'purple'])

## Load Data

In [ ]:
data_path = '/home/sebi/data/dends'

# Spikes
running_spikes = pd.read_pickle(os.path.join(data_path, 'rf_running_spikes.pkl'))
nonrunning_spikes = pd.read_pickle(os.path.join(data_path, 'rf_nonrunning_spikes.pkl'))
swr_spikes = pd.read_pickle(os.path.join(data_path, 'rf_swr_spikes.pkl'))

spikes = [running_spikes, nonrunning_spikes, swr_spikes]
spikes = [x.loc[x['condition'] == 1] for x in spikes]
for spike, s in zip(spikes, ['RUN', 'REST', 'SWR']):
    spike['state'] = s
    spike['grouped_order'] = spike.apply(group_order, axis=1)

spikes = pd.concat(spikes)
spikes_by_branch = spikes.groupby(['expt', 'mouse_name', 'fov', 'day', 'condition', 'soma', 'roi', 'order', 'grouped_order'], as_index=False).mean()

# BR
running_br = pd.read_pickle(os.path.join(data_path, 'rf_running_br.pkl'))
nonrunning_br = pd.read_pickle(os.path.join(data_path, 'rf_nonrunning_br.pkl'))
swr_br = pd.read_pickle(os.path.join(data_path, 'rf_swr_br.pkl'))
brs = [running_br, nonrunning_br, swr_br]
brs = [x.loc[x['condition'] == 1] for x in brs]
for br, s in zip(brs, ['RUN', 'REST', 'SWR']):
    br['state'] = s
    br['grouped_order'] = br.apply(group_order, axis=1)

brs = pd.concat(brs)

br = pd.read_pickle(os.path.join(data_path, 'rf_all_br.pkl'))

# Plotting

### Branch-distance characterization

In [ ]:
plotdir= '/home/sebi/plots/dends/fig2'

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

cdf(ax, br['distance_to_soma'], color='k', lw=1)

clean(ax, offset=4)
ax.set_xlabel(r'Distance to Soma ($\mu$m)')
ax.set_ylabel('Cumulative Fraction')

ax.set_xlim([0, 175])
ax.set_xticks([0, 50, 100, 150])

fig.savefig(os.path.join(plotdir, 'branch_dist_distribution.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

n, bins, _ = ax.hist(br['order'], color='k', bins=np.arange(11)-0.5, rwidth=0.75, density=True)

clean(ax, offset=1)
ax.set_xlabel(r'Branch Order')
ax.set_ylabel('Fraction of Branches')

# ax.set_xlim([0, 175])
ax.set_xticks(range(1, 10))

fig.savefig(os.path.join(plotdir, 'branch_order_distribution.svg'))

In [ ]:
fig = plt.figure(figsize=(1.5, 1.5), dpi=200)
ax = fig.add_subplot(111)

br['plot_order'] = br['order'].apply(lambda x: x if x < 7 else 7)

sns.lineplot(x='plot_order', y='distance_to_soma', data=br,
             err_style='bars', ci=68, color='k', marker='o',
             markersize=2, mfc='k', mec='none', lw=1, err_kws={'elinewidth':1})
clean(ax, offset=4)
ax.set_xlabel('Branch Order')
ax.set_ylabel('Branch Distance to Soma (um)')

ax.set_xticks(range(1, 8))
ax.set_xticklabels(['1', '2', '3', '4', '5', '6', '7+'])
ax.set_ylim([0, 85])

fig.savefig(os.path.join(plotdir, 'distance_vs_order.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

countbins = np.arange(0, 100, 10)

means, bins, _ = binned_statistic(spikes_by_branch['distance_to_soma'], spikes_by_branch['normalized_value'],
                                              statistic=np.nanmean,
                                              bins=countbins)

stds, bins, _ = binned_statistic(spikes_by_branch['distance_to_soma'], spikes_by_branch['normalized_value'],
                                          statistic=sem,
                                          bins=countbins)

color = 'k'
ax.errorbar(bins[:-1] + 5, means, yerr=stds, fmt='o', mfc='k',
            mec=color, ecolor=color, ms=2, elinewidth=1, mew=0.5,
            ls='-', color=color, lw=1)
clean(ax, offset=4)
ax.set_xlabel(r'Distance to Soma ($\mu$m)')
ax.set_ylabel('Normalized Dendritic Response')

ax.set_ylim([0, 1]);
ax.set_xlim([0, 100])
fig.savefig(os.path.join(plotdir, 'spike_size_vs_distance.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

spikes_by_branch['grouped_order'] = spikes_by_branch.apply(group_order, axis=1)
sns.lineplot(y='normalized_value', x='grouped_order', data=spikes_by_branch, err_style='bars', ci=68,
             color='k', marker='o', markersize=2, mfc='k', mec='none', lw=1, err_kws={'elinewidth':1})
clean(ax, offset=4)
ax.set_xlabel('Branch Order')
ax.set_ylabel('Normalized Dendritic Response')

ax.set_xticks(range(1, 8))
ax.set_xticklabels(['1', '2', '3', '4', '5', '6', '7+']);
ax.set_ylim([0, 1])

fig.savefig(os.path.join(plotdir, 'spike_size_vs_branch_order.svg'))

In [ ]:
# Example
expt = dbExperiment(10290)

soma_trans = expt.transientsData(roi_filter=lambda x: x.label=='cell19')
idx = soma_trans['start_indices'][0][0][1]

trans = expt.transientsData()
sigs = expt.imagingData(dFOverF='from_file')
rois = expt.rois()
rlabels = [r.label for r in rois]

plot_labels = ['cell19', 'cell19_2', 'cell19_3_0', 'cell19_2_0', 'cell19_3_1', 'cell19_2_0_0', 'cell19_3_0_0', 'cell19_3_0_1']

fig = plt.figure(figsize=(0.6, 1.5), dpi=200)
ax = fig.add_subplot(111)

pre = 10
post = 25

hm = np.zeros((len(plot_labels), pre+post))

T = np.arange(-pre, post) * expt.frame_period()

for ri, rlabel in enumerate(plot_labels):
    ridx = rlabels.index(rlabel)    
    hm[ri, :] = sigs[ridx, idx-pre:idx+post, 0]
    
sns.heatmap(hm, vmin=0, vmax=9, cbar_kws = dict(use_gridspec=False,location="left"), rasterized=True)
cbar = ax.collections[0].colorbar
cbar.set_ticks([0, 9])
cbar.set_ticklabels(['0', '900'])
cbar.set_label(r'%$\Delta$F/F')
ax.invert_yaxis()
ax.set_xticks([])
ax.set_yticks([])
        
clean(ax)
sns.despine(left=True, bottom=True)

fig.savefig(os.path.join(plotdir, 'order_size_example_hm.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

countbins = np.arange(0, 100, 10)

means, bins, _ = binned_statistic(br['distance_to_soma'], br['value'],
                                              statistic=np.nanmean,
                                              bins=countbins)

stds, bins, _ = binned_statistic(br['distance_to_soma'], br['value'],
                                          statistic=sem,
                                          bins=countbins)

color = 'k'
ax.errorbar(bins[:-1] + 5, means, yerr=stds, fmt='o', mfc='k',
            mec=color, ecolor=color, ms=2, linewidth=1, mew=0.5,
            ls='-', color=color)
clean(ax, offset=4)
ax.set_xlabel(r'Distance to Soma ($\mu$m)')
ax.set_ylabel('Dendrite-Soma Coupling')

# ax.set_xticks(range(1, 8))
# ax.set_xticklabels(['1', '2', '3', '4', '5', '6', '7+']);
ax.set_ylim([0, 1]);
ax.set_xlim([0, 100])
fig.savefig(os.path.join(plotdir, 'br_vs_distance.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

br['grouped_order'] = br.apply(group_order, axis=1)

sns.lineplot(y='value', x='grouped_order', data=br, err_style='bars', ci=68,
             color='k', marker='o', ms=2, mfc='k', mec='none', linewidth=1, mew=0.5, err_kws={'elinewidth':1})
clean(ax, offset=4)
ax.set_xlabel('Branch Order')
ax.set_ylabel('Dendrite-Soma Coupling')

ax.set_xticks(range(1, 8))
ax.set_xticklabels(['1', '2', '3', '4', '5', '6', '7+']);
ax.set_ylim([0, 1]);
fig.savefig(os.path.join(plotdir, 'br_vs_branch_order.svg'))

### Activity Characterization

In [ ]:
# Somatic Event Frequency
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

lpl.cdf(ax, running_freq.loc[running_freq['soma']]['frequency'], color='r', label='RUN', lw=1);
lpl.cdf(ax, nonrunning_freq.loc[nonrunning_freq['soma']]['frequency'], color='b', label='REST', lw=1);
lpl.cdf(ax, swr_freq.loc[swr_freq['soma']]['frequency'].dropna(), color='k', label='SWR', lw=1);

ax.set_xscale('log')

clean(ax, offset=4)
ax.set_ylabel('Cumulative Fraction')
ax.set_xlabel('Somatic Event Rate (Hz)')

ax.set_xticks([1.e-03, 1.e-02, 1.e-01])
ax.set_xticklabels(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$'])

left, bottom, width, height = [0.61, 0.2, 0.25, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])

vals = []
for val, state in zip([running_freq, nonrunning_freq, swr_freq], ['RUN', 'REST', 'SWR']):
    val = val.loc[val['soma']]
    val['state'] = state
    vals.append(val)
    
vals = pd.concat(vals)

sns.barplot(x='state', y='frequency', data=vals, palette=['r', 'b', 'k'], ci=68, capsize=0.05,
            dodge=False, edgecolor='0.2', linewidth=1, errwidth=1)

clean(ax2, offset=2)
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.set_xticks([])

fig.savefig(os.path.join(plotdir, 'soma_event_freq.svg'))


a = vals.loc[vals['state'] == 'RUN']['frequency'].dropna()
b = vals.loc[vals['state'] == 'REST']['frequency'].dropna()
c = vals.loc[vals['state'] == 'SWR']['frequency'].dropna()

print f_oneway(a, b, c)

print a.shape[0], b.shape[0], c.shape[0]

print a.mean(), a.sem()
print b.mean(), b.sem()
print c.mean(), c.sem()

vals = vals.dropna(subset=['frequency'])

print pairwise_tukeyhsd(endog=vals['frequency'], groups=vals['state'], alpha=0.05)

In [ ]:
# Somatic Amplitudes
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

colors = ['r', 'b', 'k']

tests = []

for bsp, label, color in zip([running_bsp, nonrunning_bsp, swr_bsp], ['RUN', 'REST', 'SWR'], colors):
#     tests.append(bsp)
    bsp = bsp.loc[bsp['day'].isin([1, 2, 3])]
    lpl.cdf(ax, np.log10(bsp.groupby(['expt', 'roi']).mean()['amplitude'] * 100), color=color, lw=1)
    
    bsp['state'] = label
    tests.append(bsp)

clean(ax, offset=4)
ax.set_ylabel('Cumulative Fraction')
ax.set_xlabel(r'Mean Somatic Event %$\Delta$F/F')

ax.set_xticks([1, 2, 3])
ax.set_xticklabels([10, 100, 1000])

# Inset
left, bottom, width, height = [0.6, 0.2, 0.25, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])

bsps = []
for bsp, state in zip([running_bsp, nonrunning_bsp, swr_bsp], ['RUN', 'REST', 'SWR']):
    bsp = bsp.loc[bsp['day'].isin([1, 2, 3])]
    bsp = bsp.groupby(['expt', 'roi']).mean()
    bsp['state'] = state
    bsps.append(bsp)
    
bsps = pd.concat(bsps)

sns.barplot(x='state', y='amplitude', data=bsps, palette=['r', 'b', 'k'], ci=68, capsize=0.1, errwidth=1,
            dodge=False, edgecolor='0.2', linewidth=1)

clean(ax2, offset=2)
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.set_xticks([])
ax2.set_yticks([0, 2])
ax2.set_yticklabels([0, 200])

fig.savefig(os.path.join(plotdir, 'soma_amps.svg'))

# Stats
a = bsps.loc[bsps['state'] == 'RUN']['amplitude'].dropna()
b = bsps.loc[bsps['state'] == 'REST']['amplitude'].dropna()
c = bsps.loc[bsps['state'] == 'SWR']['amplitude'].dropna()

# Simpler ANOVA test than ols interface above
print f_oneway(a, b, c)

print a.shape[0], b.shape[0], c.shape[0]

print a.mean(), a.sem()
print b.mean(), b.sem()
print c.mean(), c.sem()

print pairwise_tukeyhsd(endog=bsps['amplitude'], groups=bsps['state'], alpha=0.05)

In [ ]:
# Isolated Dend Event Frequency
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

i = -1

states = ['RUN', 'REST', 'SWR']
colors=['r', 'b', 'k']
frac_coactive = []
for df, state, color in zip([run_frac_coactive, nonrun_frac_coactive, swr_frac_coactive], states, colors):
    
    df = df.loc[df['n_trans'] > i]
    df['state'] = state
    frac_coactive.append(df)
    
    cdf(ax, df['iso_value'], color=color, lw=1)
    
clean(ax, offset=4)
ax.set_xlabel('N Isolated Events\n N All Dendrite Events')
ax.set_ylabel('Cumulative Fraction')
    
left, bottom, width, height = [0.6, 0.2, 0.25, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])

plot_df = pd.concat(frac_coactive)

sns.barplot(y='iso_value', x='state', palette=colors, data=plot_df, ci=68, capsize=0.05,
            dodge=False, edgecolor='0.2', linewidth=1, errwidth=1)
clean(ax2, offset=2)
ax2.set_ylabel('')
ax2.set_xlabel('')
ax2.set_xticks([])

fig.savefig(os.path.join(plotdir, 'frac_iso.svg'))

a = run_frac_coactive['iso_value'].dropna()
b = nonrun_frac_coactive['iso_value'].dropna()
c = swr_frac_coactive['iso_value'].dropna()

print f_oneway(a, b, c)

print a.shape[0], b.shape[0], c.shape[0]

print a.mean(), a.sem()
print b.mean(), b.sem()
print c.mean(), c.sem()

run_frac_coactive['state'] = 'RUN'
nonrun_frac_coactive['state'] = 'REST'
swr_frac_coactive['state'] = 'SWR'
vals = pd.concat([run_frac_coactive, nonrun_frac_coactive, swr_frac_coactive])

print pairwise_tukeyhsd(endog=vals['iso_value'], groups=vals['state'], alpha=0.05)

In [ ]:
# Isolated Example

def parallel_sort(a, b, key=None, reverse=False):
    # Pass in key as sort you want to be applied to a
    # b will be sorted alongside a
    
    if key is None:
        new_key = None
    else:
        new_key = lambda x: key(x[0])
    
    a, b = zip(*sorted(zip(a, b), key=new_key, reverse=reverse))
    
    return list(a), list(b)

tid = 10653
roi_label = 'Cell02_3_0_0'
event_idx = 4

cell_label = roi_label.split('_')[0]
expt = dbExperiment(tid)

roi_trans = expt.transientsData(roi_filter=lambda x: x.label==roi_label)
idx = roi_trans['start_indices'][0][0][event_idx]

sigs = expt.imagingData(dFOverF='from_file', roi_filter=lambda x: x.label.startswith(cell_label + '_'))
soma_sig = expt.imagingData(dFOverF='from_file', roi_filter=lambda x: x.label == cell_label)

rois = expt.rois(roi_filter=lambda x: x.label.startswith(cell_label + '_'))
rlabels = [r.label for r in rois]
orders = [len(x.split('_')) - 1 for x in rlabels]

_, sort_idx = parallel_sort(orders, range(len(rlabels)))
sort_idx = np.array(sort_idx)

other_soma_sigs = expt.imagingData(dFOverF='from_file', roi_filter = lambda x: ('_' not in x.label) and (x.label != cell_label))

fig = plt.figure(figsize=(0.6, 1.5), dpi=200)
ax = fig.add_subplot(111)

pre = 10
post = 25

# add two for soma and other somata
hm = np.zeros((sigs.shape[0] + 2, pre+post))

T = np.arange(-pre, post) * expt.frame_period()

                                   
hm[0, :] = np.nanmean(other_soma_sigs, axis=0)[idx-pre:idx+post, 0]
hm[1, :] = soma_sig[0, idx-pre:idx+post, 0]
ri = 2
for rlabel, sig in zip(rlabels, sigs[sort_idx, ...]):
    hm[ri, :] = sig[idx-pre:idx+post, 0]
    ri +=1
    
sns.heatmap(np.nan_to_num(hm), vmin=0, vmax=4.5, cbar_kws = dict(use_gridspec=False,location="left"), rasterized=True)


# FORMATTING

cbar = ax.collections[0].colorbar
cbar.set_ticks([0, 4.5])
cbar.set_ticklabels(['0', '450'])
cbar.set_label(r'%$\Delta$F/F')
ax.invert_yaxis()
ax.set_xticks([])
ax.set_yticks([])
        
clean(ax)
sns.despine(left=True, bottom=True)

fig.savefig(os.path.join(plotdir, 'iso_example_hm.svg'))

In [ ]:
branch_spikes = da.all_branch_spikes(grp)

### Supplementary Activity Characterization

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

lpl.cdf(ax, all_frac_coactive['freq_iso'], color='k', lw=1)
lpl.cdf(ax, all_frac_coactive['freq_coincident'], color='r', lw=1)

ax.set_xlim([0, 0.02])
ax.set_xlabel('Event Rate (Hz)')
ax.set_ylabel('Cumulative Fraction')

print spearmanr(all_frac_coactive['freq_iso'], all_frac_coactive['freq_coincident'])
print wilcoxon(all_frac_coactive['freq_iso'], all_frac_coactive['freq_coincident'])

# fig.savefig(os.path.join(plotdir, 'dend_freq.svg'))

In [ ]:
print all_frac_coactive['freq_iso'].mean(), all_frac_coactive['freq_iso'].sem()
print all_frac_coactive['freq_coincident'].mean(), all_frac_coactive['freq_coincident'].sem()

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

all_dend_freq = all_freq.loc[~all_freq['soma']]
all_soma_freq = all_freq.loc[all_freq['soma']]

lpl.cdf(ax, all_dend_freq['frequency'], color='0.5', lw=1)
lpl.cdf(ax, all_soma_freq['frequency'], color='b', lw=1)

ax.set_xlim([0, 0.05])

ax.set_xlabel('Event Rate (Hz)')
ax.set_ylabel('Cumulative Fraction')

print mannwhitneyu(all_dend_freq['frequency'], all_soma_freq['frequency'])

# fig.savefig(os.path.join(plotdir, 'soma_v_dend_freq.svg'))

In [ ]:
print all_dend_freq['frequency'].mean(), all_dend_freq['frequency'].sem()
print all_soma_freq['frequency'].mean(), all_soma_freq['frequency'].sem()

In [ ]:
def amplitudes(grp, by_cell=True):
    
    data_list = []
    
    for expt in grp:
        
        tid = expt.trial_id
        
        rois = expt.rois()
        trans = expt.transientsData()['max_amplitudes']
        sigma = expt.transientsData()['sigma']
        
        for r,t, s in zip(rois, trans, sigma):
            
            soma = '_' not in r.label
            
            if by_cell:
            
                mean_amp = np.nanmean(t[0])
                data_list.append({'expt': tid,
                                  'roi': r.label,
                                  'mean_amp': mean_amp,
                                  'soma': soma})
            
            else:
                for amp in t[0]:
                    data_list.append({'expt': tid,
                                  'roi': r.label,
                                  'amp': amp,
                                  'soma': soma,
                                  'zamp': amp / s})
    
    return pd.DataFrame(data_list)

all_amps = amplitudes(grp, by_cell=False)

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

#TODO separate isolated vs coincident dendritic spikes?

all_dend_amps = all_amps.loc[~all_amps['soma']]
all_soma_amps = all_amps.loc[all_amps['soma']]

lpl.cdf(ax, all_dend_amps['mean_amp'].dropna()*100, color='0.5', lw=1);
lpl.cdf(ax, all_soma_amps['mean_amp'].dropna()*100, color='b', lw=1);

ax.set_xlim([0, 1000])
ax.set_xlabel(r'Mean Event %$\Delta$F/F')

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

all_dend_amps = all_amps.loc[~all_amps['soma']]
all_soma_amps = all_amps.loc[all_amps['soma']]

lpl.cdf(ax, np.log10(all_dend_amps['amp'].dropna()*100), color='0.5', lw=1);
lpl.cdf(ax, np.log10(all_soma_amps['amp'].dropna()*100), color='b', lw=1);

ax.set_xlim([1, 3.3])
ax.set_xticks([1, 2, 3])
ax.set_xticklabels([10, 100, 1000])
ax.set_xlabel(r'Event %$\Delta$F/F')
ax.set_ylabel('Cumulative Fraction')

fig.savefig(os.path.join(plotdir, 'soma_v_dend_amps.svg'))

In [ ]:
a = all_dend_amps['amp'].dropna()*100
print a.shape, a.mean(), a.sem()
b = all_soma_amps['amp'].dropna()*100
print b.shape, b.mean(), b.sem()
test = mannwhitneyu(a, b)

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

all_iso_amps = branch_spikes.loc[~branch_spikes['coincident']]
all_co_amps = branch_spikes.loc[branch_spikes['coincident']]

lpl.cdf(ax, np.log10(all_iso_amps['amp'].dropna()*100), color='k', lw=1);
lpl.cdf(ax, np.log10(all_co_amps['amp'].dropna()*100), color='r', lw=1);

ax.set_xlim([1, 3.3])
ax.set_xticks([1, 2, 3])
ax.set_xticklabels([10, 100, 1000])
ax.set_xlabel(r'Event %$\Delta$F/F')
ax.set_ylabel('Cumulative Fraction')

fig.savefig(os.path.join(plotdir, 'iso_v_co_dend_amps.svg'))

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

all_iso_amps = branch_spikes.loc[~branch_spikes['coincident']]
all_co_amps = branch_spikes.loc[branch_spikes['coincident']]

# Filter low amplitude to address reviewer comment
all_co_amps = all_co_amps.loc[all_co_amps['zamp'] > 5]

lpl.cdf(ax, np.log10(all_iso_amps['amp'].dropna()*100), color='k', lw=1);
lpl.cdf(ax, np.log10(all_co_amps['amp'].dropna()*100), color='r', lw=1);

ax.set_xlim([1, 3.3])
ax.set_xticks([1, 2, 3])
ax.set_xticklabels([10, 100, 1000])
ax.set_xlabel(r'Event %$\Delta$F/F')
ax.set_ylabel('Cumulative Fraction')

# fig.savefig(os.path.join(plotdir, 'iso_v_co_dend_amps.svg'))

In [ ]:
# If we only include >5sigma events
all_frac_coactive = da.branch_impact(grp, signal=None)

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

lpl.cdf(ax, all_frac_coactive['freq_solo'], color='k', lw=1, label='Isolated')
lpl.cdf(ax, all_frac_coactive['freq_coactive'], color='r', lw=1, label='Coincident')

ax.set_xlim([0, 0.02])
ax.set_xlabel('Event Rate (Hz)')
ax.set_ylabel('Cumulative Fraction')

ax.set_title(r'Equal Thresholds for Event Types (5$\sigma$)')

ax.legend(fontsize=6, loc='upper left', bbox_to_anchor=(0.5, 0.5))

print spearmanr(all_frac_coactive['freq_solo'], all_frac_coactive['freq_coactive'])
print wilcoxon(all_frac_coactive['freq_solo'], all_frac_coactive['freq_coactive'])

# fig.savefig(os.path.join(plotdir, 'dend_freq.svg'))

In [ ]:
print all_frac_coactive['freq_solo'].mean()
print all_frac_coactive['freq_solo'].sem()

print all_frac_coactive['freq_coactive'].mean()
print all_frac_coactive['freq_coactive'].sem()

In [ ]:
a = all_iso_amps['amp'].dropna()*100
print a.shape, a.mean(), a.sem()
b = all_co_amps['amp'].dropna()*100
print b.shape, b.mean(), b.sem()
print mannwhitneyu(a, b)

### Excitability

In [ ]:
# Precomputed in analysis-scripts/state_psth.py

means = []
sems = []

fig = plt.figure(figsize=(1.5, 1))
ax = fig.add_subplot(111)

start = 7
stop = -7

for fname, color in zip(['running', 'nonrunning', 'swr'], ['r', 'b', 'k']):

    with open('/home/sebi/psth_{}.pkl'.format(fname), 'rb') as fp:
        T, psth = pkl.load(fp)

    psth_mean = np.nanmean(psth, axis=0)
    
    print psth.shape[0], np.max(psth_mean)

    filtered_mean = gaussian_filter1d(psth_mean, 2)[start:stop]
    psth_sem = sem(psth, axis=0, nan_policy='omit')
    psth_sem = gaussian_filter1d(psth_sem, 2)[start:stop]
    
    T = T[start:stop]
    
    ax.plot(T, filtered_mean, color=color)
    ax.fill_between(T, y1=filtered_mean - psth_sem, y2=filtered_mean + psth_sem, alpha=0.25, color=color)

ax.axvline(0, color='r', ls='--')

clean(ax)

# fig.savefig('/home/sebi/plots/dends/state_psth.svg')

In [ ]:
# Responsiveness
dspikes = [running_spikes, nonrunning_spikes, swr_spikes]
dspikes = [x.loc[x['condition'] == 1] for x in dspikes]

fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

min_bin = max([np.nanpercentile(dspike['soma_spike_amp'], 0.5) for dspike in dspikes])
max_bin = min([np.nanpercentile(dspike['soma_spike_amp'], 99.5) for dspike in dspikes])

n_bins = 8

colors = ['r', 'b', 'k']

for dspike, color in zip(dspikes, colors):
    means, bins, _ = binned_statistic(dspike['soma_spike_amp'], dspike['value'],
                                              statistic=np.mean,
                                              bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

    stds, bins, _ = binned_statistic(dspike['soma_spike_amp'], dspike['value'],
                                              statistic=sem,
                                              bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

    log_bins = np.log10(bins)
    plot_bins = 10 ** (log_bins[:-1] + 0.5 * (log_bins[1] - log_bins[0]))

    # color = sns.xkcd_palette(['pumpkin'])[0]
    ax.errorbar(np.log10(plot_bins * 100), means * 100, yerr=stds * 100, fmt='o', mfc=color, mec=color,
                ecolor=color, ms=2, elinewidth=1, mew=0.5, ls='-', color=color, lw=1)

# ax.set_xscale('log')

# sns.scatterplot(y='value', x='amplitude', data=context_bsp)
clean(ax, offset=4)
ax.set_xticks(np.log10([50, 100, 200, 400]))
ax.set_xticklabels(['50', '100', '200', '400'])
ax.set_xlabel(r'Somatic %$\Delta$F/F')
ax.set_ylabel(r'Dendritic %$\Delta$F/F')

fig.savefig(os.path.join(plotdir, 'spike_size_by_state_all_dends_even_failures.svg'))

#Bootstrap CI test
# shuffle_means = []

# seed = 14230
# for df in dspikes:
# #     df['logdf'] = df['amplitude'].apply(np.log10)

#     n_samples = df.shape[0]
#     n_shuffles = 1000

#     shuffle_formula="value ~ soma_spike_amp + 0"

#     state_means = []
#     for _ in xrange(n_shuffles):
#         shuffle = df.sample(n=n_samples, replace=True, random_state=seed)
#         seed += 1
#         shuffle_lm = ols(shuffle_formula, shuffle).fit()

#         state_means.append(shuffle_lm.params['soma_spike_amp'])
    
#     shuffle_means.append(state_means)

# for state_means in shuffle_means:
#     print np.percentile(state_means, 2.5), np.percentile(state_means, 97.5)

In [ ]:
# Branch Spike Prevalence

bsps = [running_bsp, nonrunning_bsp, swr_bsp]
bsps = [x.loc[x['condition'] == 1] for x in bsps]

fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

min_bin = max([np.nanpercentile(bsp['amplitude'], 0.5) for bsp in bsps])
max_bin = min([np.nanpercentile(bsp['amplitude'], 99.5) for bsp in bsps])

n_bins = 9

colors = ['r', 'b', 'k']

for bsp, color in zip(bsps, colors):
    means, bins, _ = binned_statistic(bsp['amplitude'], bsp['value'],
                                              statistic=np.mean,
                                              bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

    stds, bins, _ = binned_statistic(bsp['amplitude'], bsp['value'],
                                              statistic=sem,
                                              bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

    log_bins = np.log10(bins)
    plot_bins = 10 ** (log_bins[:-1] + 0.5 * (log_bins[1] - log_bins[0]))

    ax.errorbar(np.log10(plot_bins * 100), means, yerr=stds, fmt='o', mfc=color,
                mec=color, ecolor=color, ms=2, elinewidth=1, mew=0.5, lw=1,
                ls='-', color=color)

clean(ax, offset=4)
ax.set_xticks(np.log10([50, 100, 200, 400]))
ax.set_xticklabels(['50', '100', '200', '400'])
ax.set_xlabel(r'Somatic %$\Delta$F/F')
ax.set_ylabel('Branch Spike Prevalence')

fig.savefig(os.path.join(plotdir, 'bsp_by_state.svg'))

#Bootstrap CI test

# shuffle_means = []

# seed = 11230
# for df in bsps:
#     df['logdf'] = df['amplitude'].apply(np.log10)

#     n_samples = df.shape[0]
#     n_shuffles = 1000

#     shuffle_formula="value ~ amplitude + 0"

#     state_means = []
#     for _ in xrange(n_shuffles):
#         shuffle = df.sample(n=n_samples, replace=True, random_state=seed)
#         seed += 1
#         shuffle_lm = ols(shuffle_formula, shuffle).fit()

#         state_means.append(shuffle_lm.params['amplitude'])
    
#     shuffle_means.append(state_means)

# for state_means in shuffle_means:
#     print np.percentile(state_means, 2.5), np.percentile(state_means, 97.5)

In [ ]:
# Failure Fraction
bsps = [running_bsp, nonrunning_bsp, swr_bsp]
bsps = [x.loc[x['condition'] == 1] for x in bsps]

fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

min_bin = max([np.nanpercentile(bsp['amplitude'], 0.5) for bsp in bsps])
max_bin = min([np.nanpercentile(bsp['amplitude'], 99.5) for bsp in bsps])

n_bins = 9

colors = ['r', 'b', 'k']

def failure_rate(x):
    
    return np.sum(x == 0) / float(len(x))

for bsp, color in zip(bsps, colors):
    means, bins, _ = binned_statistic(bsp['amplitude'], bsp['value'],
                                              statistic=failure_rate,
                                              bins=np.logspace(np.log10(min_bin),np.log10(max_bin), n_bins))

    log_bins = np.log10(bins)
    plot_bins = 10 ** (log_bins[:-1] + 0.5 * (log_bins[1] - log_bins[0]))

    ax.plot(np.log10(plot_bins * 100), means, marker='o', mfc=color,
                mec=color, ms=2, mew=0.5,
                ls='-', color=color, lw=1)

clean(ax, offset=4)
ax.set_xticks(np.log10([50, 100, 200, 400]))
ax.set_xticklabels(['50', '100', '200', '400'])
ax.set_xlabel(r'Somatic %$\Delta$F/F')
ax.set_ylabel('BSP Failure Rate')


fig.savefig(os.path.join(plotdir, 'failure_by_state.svg'))

# shuffle_vals = []

# seed=1776

# bsps = [running_bsp, nonrunning_bsp, swr_bsp]
# for bsp, state, color in zip(bsps, ['RUN', 'REST', 'SWR'], ['r', 'b', 'k']):
#     bsp['success'] = bsp['value'].apply(lambda x: x > 0).astype(float)
    
#     n_samples = bsp.shape[0]
#     n_shuffles = 1000
    
#     state_vals = []
#     for _ in xrange(n_shuffles):
#         shuffle = bsp.sample(n=n_samples, replace=True, random_state=seed)
#         seed += 1

#         ytrain = np.stack([shuffle['amplitude'], np.ones(shuffle['amplitude'].shape)]).T

#         logit = sm.Logit(shuffle['success'], ytrain).fit(disp=0)
#         state_vals.append(logit.params['x1'])
    
#     shuffle_vals.append(state_vals)

# for state_vals in shuffle_vals:
#     print np.nanpercentile(state_vals, 2.5), np.nanpercentile(state_vals, 97.5)

### Branch correlations

In [ ]:
fig = plt.figure(figsize=(1, 1.5), dpi=200)
ax = fig.add_subplot(111)

coactivity['plot_dist'] = coactivity['order_dist'].apply(lambda x: x if x < 13 else 13)
sns.lineplot(y='corr', x='plot_dist', data=coactivity, err_style='bars', color='b', ci=68,
             lw=1, err_kws={'elinewidth':1}, ms=2, marker='o', mec='none')
sns.lineplot(y='partial_corr', x='plot_dist', data=coactivity, err_style='bars', color='k',
             ci=68, lw=1, err_kws={'elinewidth':1}, ms=2, marker='o', mec='none',)
# sns.lineplot(y='dend_fraction', x='order_dist', data=nonrun_coactivity, err_style='bars', color='b', ci=68)
# sns.lineplot(y='dend_fraction', x='order_dist', data=swr_coactivity, err_style='bars', color='k')
# ax.set_xlim([1, 12])
ax.set_ylim([0, .3])
ax.set_ylabel(r'$\Delta$F/F Correlation')
ax.set_xlabel('Distance (N Bifurcations)')

ax.set_xticks(range(1, 15, 2))
ax.set_xticklabels(['1', '3', '5', '7', '9', '11', '13+'])
ax.set_yticks([0, 0.1, 0.2, 0.3])
# ax.set_xticklabels(['0', '2', '4', '6', '8', '10', '12+'])

clean(ax, offset=4)

fig.savefig(os.path.join(plotdir, 'correlation_vs_order_distance.svg'))

## Compute/Save Data

In [ ]:
data_path = '/home/sebi/data/dends'

running_bsp = da.branch_spike_prevalence(grp, signal=None, running_only=True)
# in_pf_bsp = da.branch_spike_prevalence(grp, signal=None, running_only=True, in_field_only=True)
# out_pf_bsp = da.branch_spike_prevalence(grp, signal=None, running_only=True, out_field_only=True)
nonrunning_bsp = da.branch_spike_prevalence(grp, signal=None, non_running_only=True, non_ripple_only=True)
swr_bsp = da.branch_spike_prevalence(grp, signal=None, non_running_only=True, ripple_only=True)
# nonrunning_bsp.to_pickle(os.path.join(data_path, 'rf_nonrunning_bsp.pkl'))
# swr_bsp.to_pickle(os.path.join(data_path, 'rf_swr_bsp.pkl'))

# running_br = da.branch_recruitment(grp, signal=None, running_only=True)
# in_pf_br = da.branch_recruitment(grp, signal=None, running_only=True, in_field_only=True)
# out_pf_br = da.branch_recruitment(grp, signal=None, running_only=True, out_field_only=True)
# nonrunning_br = da.branch_recruitment(grp, signal=None, non_running_only=True, non_ripple_only=True)
# swr_br = da.branch_recruitment(grp, signal=None, non_running_only=True, ripple_only=True)
# nonrunning_br.to_pickle(os.path.join(data_path, 'rf_nonrunning_br.pkl'))
# swr_br.to_pickle(os.path.join(data_path, 'rf_swr_br.pkl'))

# running_br = pd.read_pickle(os.path.join(save_path, 'rf_running_br.pkl'))
# nonrunning_br = pd.read_pickle(os.path.join(save_path, 'rf_nonrunning_br.pkl'))
# swr_br = pd.read_pickle(os.path.join(save_path, 'rf_swr_br.pkl'))


# running_spikes = da.branch_spikes(grp, signal=None, running_only=True, include_failures=True)
# in_pf_spikes = da.branch_spikes(grp, signal=None, running_only=True, in_field_only=True)
# out_pf_spikes = da.branch_spikes(grp, signal=None, running_only=True, out_field_only=True)
# nonrunning_spikes = da.branch_spikes(grp, signal=None, non_running_only=True, non_ripple_only=True, include_failures=True)
# swr_spikes = da.branch_spikes(grp, signal=None, non_running_only=True, ripple_only=True, include_failures=True)
# nonrunning_spikes.to_pickle(os.path.join(data_path, 'rf_nonrunning_spikes.pkl'))
# swr_spikes.to_pickle(os.path.join(data_path, 'rf_swr_spikes.pkl'))

In [ ]:
br = da.branch_recruitment(grp, signal=None)

In [ ]:
running_freq = da.frequency(grp, signal=None, running_only=True)
nonrunning_freq = da.frequency(grp, signal=None, non_running_only=True, non_ripple_only=True)
swr_freq = da.frequency(grp, signal=None, non_running_only=True, ripple_only=True)

In [ ]:
all_freq = da.frequency(grp, signal=None)

In [ ]:
all_freq.to_pickle(os.path.join(data_path, 'rf_all_freq.pkl'))

In [ ]:
running_freq.to_pickle(os.path.join(data_path, 'rf_run_freq.pkl'))

In [ ]:
run_frac_coactive = da.branch_impact(grp, signal=None, running_only=True)
nonrun_frac_coactive = da.branch_impact(grp, signal=None, non_running_only=True, non_ripple_only=True)
swr_frac_coactive = da.branch_impact(grp, signal=None, non_running_only=True, ripple_only=True)

In [ ]:
all_frac_coactive = da.branch_impact(grp, signal=None)

In [ ]:
nonrun_frac_coactive.to_pickle(os.path.join(data_path, 'rf_nonrun_frac_coactive.pkl'))

In [ ]:
coactivity = da.branch_coactivation(grp, signal=None)

In [ ]:
def get_order_dist(labels):

    u = labels[0].lower().split('_')
    v = labels[1].lower().split('_')

    shorter = min(len(u), len(v))
    longer = max(len(u), len(v))
    
    for i in xrange(shorter):
        if u[i] != v[i]:
            i = i-1
            break

    if i == shorter - 1:
        return longer - i - 1
    else:
        return (longer - i - 1) + (shorter - i - 1)

coactivity['order_dist'] = coactivity['roi_pair'].apply(get_order_dist)

In [ ]:
epochs = da.interval_durs(grp)

In [ ]:
epochs.mean()